# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'company/brand page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'Community/Discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'API Endpoints page', 'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
11 days ago
•
330k
•
3.07k
nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16
Updated
about 20 hours ago
•
62.5k
•
373
tencent/HY-WorldPlay
Updated
1 day ago
•
340
XiaomiMiMo/MiMo-V2-Flash
Updated
1 day ago
•
3.63k
•
301
ResembleAI/chatterbox-turbo
Updated
4 days ago
•
275
Browse 2M+ models
Spaces
Running
on
Zero
287
TRELLIS.2
🏢
287
High-fidelity 3D Generation from images
Running
on
Zero
MCP
Featured
286
Chatterbox Turbo Demo
⚡
286
Chatterbox Turbo Demo
Running
on
Zero
700
Z Image Turbo
🖼
700
Generate stunning

In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 18 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n11 days ago\n•\n330k\n•\n3.07k\nnvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16\nUpdated\nabout 20 hours ago\n•\n62.5k\n•\n373\ntencent/HY-WorldPlay\nUpdated\n1 day ago\n•\n340\nXiaomiMiMo/MiMo-V2-Flash\nUpdated\n1 day ago\n•\n3.63k\n•\n301\nResembleAI/chatterbox-turbo\nUpdated\n4 days ago\n•\n275\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\n287\nTRELLIS.2\n🏢

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face: Building the Future of AI Together

---

## Who We Are  
Hugging Face is the leading collaboration platform for the global machine learning (ML) community. We serve as the central hub where researchers, engineers, scientists, and AI enthusiasts come together to create, share, and experiment with machine learning models, datasets, and applications—building the open and ethical AI future of tomorrow.

Our platform supports all AI modalities—text, images, audio, video, and even 3D—empowering creators to explore the full potential of machine learning.

---

## What We Offer

- **2M+ Machine Learning Models**  
  Huge library of models openly accessible for diverse AI tasks, from natural language processing to computer vision and beyond.

- **500k+ Datasets**  
  Curated datasets supporting cutting-edge research and development optimized for collaboration and discovery.

- **Spaces: Share & Deploy ML Apps**  
  Host and showcase interactive ML applications in one place, fostering community feedback and iterative innovation.

- **Open Source Tools & Libraries**  
  Powered by a fast-growing open-source ecosystem, enabling faster experimentation and deployment.

- **Enterprise Solutions**  
  Paid compute resources and tailored enterprise offerings enable organizations to build AI at scale securely and efficiently.

---

## Our Community Culture

- **Open & Collaborative**  
  We unlock innovation through open sharing of models, data, and research. Everyone is encouraged to contribute, learn, and grow.

- **Ethical AI Focus**  
  We're committed to building and promoting AI technologies responsibly, centering transparency and inclusivity.

- **Learning & Growth**  
  From beginners to experts, our resources, docs, and community support empower individuals to accelerate their ML journeys.

- **Global & Diverse**  
  Hugging Face unites a vibrant international community of ML practitioners, researchers, and developers.

---

## For Our Customers & Partners

Hugging Face provides the most advanced platform for teams and enterprises who want to:

- Accelerate AI development with state-of-the-art models and datasets  
- Collaborate securely with dedicated tools and compute resources  
- Build custom AI solutions tailored to industry needs  
- Leverage open source innovation with enterprise-grade support  

Trusted by top AI practitioners, companies, and research institutions worldwide, Hugging Face is where AI breakthroughs happen.

---

## Careers at Hugging Face

Join us to be part of an inspiring team shaping the AI revolution. We are looking for passionate individuals who:

- Are excited about open source and cutting-edge AI technologies  
- Thrive in a collaborative, inclusive, and fast-moving environment  
- Want to impact millions through technology that empowers and educates  
- Value ethical innovation and community-driven development  

Explore current openings on our [Careers Page](https://huggingface.co/careers) and become a vital part of the AI community building the future.

---

## Connect & Learn More

- Visit our website: [huggingface.co](https://huggingface.co)  
- Join our vibrant community on GitHub, Twitter, LinkedIn, and Discord  
- Explore our blog for the latest research, case studies, and insights  
- Access comprehensive documentation and tutorials to get started  

---

Hugging Face — **Empowering the next generation of machine learning engineers, scientists, and creators to build a better AI future, together.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 18 relevant links


# Hugging Face Brochure

---

## Who We Are

**Hugging Face** is the vibrant AI community building the future of machine learning (ML). We provide the central collaboration platform where ML engineers, scientists, and enthusiasts come together to create, share, and grow open-source models, datasets, and applications. Our mission is to empower the next generation of AI practitioners by enabling easy access to tools and resources in an open, ethical, and community-driven environment.

---

## What We Offer

### A Hub for Machine Learning Innovation
- **2M+ Models**: Explore the largest collection of machine learning models for tasks across text, image, video, audio, and even 3D.
- **500k+ Datasets**: Access vast, diverse datasets to fuel your AI experiments and research.
- **1M+ Applications**: Browse and deploy ready-to-use AI-powered applications or create your own.
- **Spaces**: Host and run machine learning demos and applications within zero-config environments for collaboration and sharing.
  
### Collaboration and Open Source Stack
- Share your projects publicly to build your portfolio and gain worldwide recognition.
- Collaborate on models, datasets, and AI applications seamlessly.
- Use cutting-edge open-source tools to accelerate model building, fine-tuning, and deployment.

### Enterprise Solutions
- Paid compute resources to scale AI projects efficiently.
- Advanced team and enterprise platforms for businesses looking to harness AI with robust, secure, and scalable tools.

---

## Our Customers and Community

Hugging Face supports a fast-growing global community including:
- AI researchers and ML engineers pushing the boundaries of artificial intelligence.
- Enterprises adopting AI with scalable and customizable AI infrastructure.
- Open source contributors sharing innovations in natural language processing, computer vision, audio processing, and more.
- Educators and learners building skills with accessible and collaborative tools.

Our users benefit from continuous updates and collaborative discovery, with trending models such as NVIDIA’s Nemotron, Tencent’s HY-WorldPlay, and Xiaomi’s MiMo-V2 flash inspiring creativity and innovation.

---

## Company Culture & Values

- **Open and Ethical AI**: We believe in transparency, openness, and ethical standards to shape AI that benefits everyone.
- **Community-Driven**: Our platform thrives on the contributions and collaboration of a global ML community.
- **Innovation at Speed**: We enable rapid prototyping and scaling of AI solutions thanks to our open-source stack and cloud resources.
- **Diversity and Inclusion**: We welcome participants from all backgrounds to join and contribute to a thriving, diverse AI ecosystem.

---

## Careers at Hugging Face

Join a passionate and growing team dedicated to democratizing AI:
- Work alongside leading AI researchers, engineers, and open-source contributors.
- Build tools and platforms that impact millions worldwide.
- Engage in a culture that values collaboration, creativity, and continuous learning.
- Opportunities range across software development, ML research, product management, and community building.

---

## Get Involved

- **Explore Models & Datasets**: Dive into the Hugging Face Hub and start experimenting.
- **Join the Community**: Connect via forums and collaboration spaces.
- **Build Your Portfolio**: Share your projects and establish your presence in the AI community.
- **Enterprise Solutions**: Contact us to empower your team with scalable AI infrastructure.

---

### Contact & Links

- Website: [https://huggingface.co](https://huggingface.co)
- Sign Up & Start Building Today!

---

## Hugging Face — The AI community building the future.  
Create, discover, and collaborate on machine learning better, together.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>